**SQL**

**Цель и задачи исследования.**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузим и исследуем таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Исследуем таблицу books с данными о книгах

In [2]:
query = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)

books.head()


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


### Исследуем таблицу authors с данными об авторах

In [4]:
query = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)

authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


### Исследуем таблицу publishers с  данных об издательствах

In [6]:
query = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Исследуем таблицу ratings с данными о пользовательских оценках

In [8]:
query = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### Исследуем таблицу reviews с данными о пользовательских обзорах на книги

In [10]:
query = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


С данными все впорядке.

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [12]:
df_1  = '''
            SELECT 
                COUNT(book_id) as cnt
            FROM 
                books
                WHERE publication_date >= '2000-01-01'
       '''
pd.read_sql(df_1, con = engine)

,cnt
0,821


После 01.01.2000 года вышло 821 книга.

## Посчитаем для каждой книги количество обзоров и среднюю оценку

In [13]:
df_2  = '''
            SELECT 
                books.book_id AS id, 
                books.title AS book_name, 
                COUNT(DISTINCT reviews.review_id) as reviews,  
                AVG(ratings.rating) as avg_rating     
            FROM 
                books
            LEFT JOIN ratings ON ratings.book_id = books.book_id 
            LEFT JOIN reviews ON reviews.book_id = books.book_id 
            GROUP BY
                books.book_id
            ORDER BY
                reviews DESC 
            LIMIT
               10
             
'''
pd.read_sql(df_2, con = engine)

,id,book_name,reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


Больше всего обзоров у книги Twilight (Twilight #1)

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [14]:
df_3  = '''
            SELECT
                publishers.publisher,
                COUNT(books.book_id) AS cnt
            FROM    
                books
            LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE 
                num_pages > 50
            GROUP BY
                publishers.publisher
            ORDER BY
                cnt DESC
            LIMIT
                10
       '''
pd.read_sql(df_3, con = engine)

,publisher,cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Больше всего книг толше 50 страниц выпустило издательство Penguin Books в кольчестве 42 книги.

## Определим автора с самой высокой средней оценкой книг — учитем только книги с 50 и более оценками

In [15]:
df_4 = '''
SELECT
      SUBQ.author AS author,
      AVG(SUBQ.avg_rating) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''
pd.read_sql(df_4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый высокий рейтинг у автора J.K. Rowling/Mary GrandPré с 50 и более оценками

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
df_5 = ''' 
    SELECT
        AVG(Sub.n_reviews)
    FROM
        (SELECT
            username,
            COUNT(review_id) AS n_reviews
    FROM
        reviews
    WHERE 
        username IN (
                SELECT 
                    username
                FROM 
                    ratings
                GROUP BY 
                    username
                HAVING 
                    COUNT(rating_id) > 50
                    )
    GROUP BY 
        username
    ) AS Sub
; ''' 
pd.read_sql(df_5, con = engine).round()

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили боьше 50 оценок = 24

## Вывод

Выполнив требуемые задания мы выяснили что:
- После 01.01.2000 года вышло 821 книга;
- Больше всего обзоров у книги Twilight (Twilight #1);
- Больше всего книг толше 50 страниц выпустило издательство Penguin Books в кольчестве 42 книги;
- Самый высокий рейтинг у автора J.K. Rowling/Mary GrandPré с 50 и более оценками;
- Среднее количество обзоров от пользователей, которые поставили боьше 50 оценок = 24.